In [85]:
import requests
from bs4 import BeautifulSoup 


In [109]:
#Check whether given website allows web scraping or not, If out 200 then good to go
r=requests.get("https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=")

r.status_code

200

In [91]:
import pandas as pd
import numpy as np
from time import sleep
from random import randint


In [92]:
#Declaring the headers 
headers = {"Accept-Language": "en-US,en;q=0.5"}

In [93]:
movie_name = []
year = []
time = []
ratings = []
metascore = []
gross = []
votes = []
description = []

In [94]:
#creating an array of values and passing it in the url for dynamic webpages
pages = np.arange(1,1000,100)

In [95]:
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start="+str(page)+"&ref_=adv_nxt")
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    sleep(randint(2,8))

In [96]:
for stored in movie_data:
  name = stored.h3.a.text
  movie_name.append(name)

  y = stored.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.replace('(', '').replace(')', '')
  year.append(y)
    
  runtime = stored.p.find('span', class_ = 'runtime').text.replace(' min', '')
  time.append(runtime)
    
  rate = stored.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n', '')
  ratings.append(rate)
    
  meta  = stored.find('span', class_ = 'metascore').text.replace(' ', '') if stored.find('span', class_ = 'metascore') else '#####'
  metascore.append(meta)
  #since, gross and votes have same attributes, so created a common variable and then used indexing
  value = stored.find_all('span', attrs = {'name': 'nv'})
    
  vote = value[0].text
  votes.append(vote)
    
  grosses = value[1].text if len(value) >1 else '*****'
  gross.append(grosses)

  describe = stored.find_all('p', class_ = 'text-muted')
  description_ = describe[1].text.replace('\n', '') if len(describe) >1 else '*****'
  description.append(description_)

In [97]:
#creating dataframe 
a ={'Name of movie': movie_name, 'Year of release': year, 'Watchtime': time, 'Movie Rating': ratings, 'Metascore': metascore, 'Votes': votes, 'Gross collection': gross, 'Description': description}
df = pd.DataFrame.from_dict(a, orient='index')
df = df.transpose()

In [98]:
df.head()

,Name of movie,Year of release,Watchtime,Movie Rating,Metascore,Votes,Gross collection,Description
0,The Florida Project,2017,111,7.6,92,"103,288",$5.90M,"Set over one summer, the film follows precocio..."
1,Just Mercy,2019,137,7.6,68,"55,982",*****,World-renowned civil rights defense attorney B...
2,Gifted,2017,101,7.6,60,"107,276",$24.80M,"Frank, a single man raising his child prodigy ..."
3,The Peanut Butter Falcon,2019,97,7.6,70,"81,739",$13.12M,Zak runs away from his care home to make his d...
4,Victoria,II 2015,138,7.6,77,"55,835",*****,A young Spanish woman who has recently moved t...


In [99]:
#change to csv
df.to_csv('movies.csv')

In [100]:
data = pd.read_csv('movies.csv')
data.shape

(100, 9)

In [101]:
#save dataframe to sql database
import sqlite3

In [102]:
conn = sqlite3.connect('TestDB.db')
c = conn.cursor()

In [103]:
c.execute('CREATE TABLE IF NOT EXISTS movies (Name of movie, Year of release, Watchtime, Movie Rating, Metascore, Vote, Gross collection, Description)')
conn.commit()

In [104]:
data.to_sql('movies', conn, if_exists = 'replace', index = False)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [105]:
c.execute('''SELECT * FROM movies''')

In [106]:
#for row in c.fetchall():
  #print(row)

In [107]:
c.execute('''SELECT * FROM movies limit 5''')

In [108]:
for row in c.fetchall():
  print(row)

(0, 'The Florida Project', '2017', 111, 7.6, '92', '103,288', '$5.90M', 'Set over one summer, the film follows precocious six-year-old Moonee as she courts mischief and adventure with her ragtag playmates and bonds with her rebellious but caring mother, all while living in the shadows of Walt Disney World.')
(1, 'Just Mercy', '2019', 137, 7.6, '68', '55,982', '*****', 'World-renowned civil rights defense attorney Bryan Stevenson works to free a wrongly condemned death row prisoner.')
(2, 'Gifted', '2017', 101, 7.6, '60', '107,276', '$24.80M', 'Frank, a single man raising his child prodigy niece Mary, is drawn into a custody battle with his mother.')
(3, 'The Peanut Butter Falcon', '2019', 97, 7.6, '70', '81,739', '$13.12M', 'Zak runs away from his care home to make his dream of becoming a wrestler come true.')
(4, 'Victoria', 'II 2015', 138, 7.6, '77', '55,835', '*****', 'A young Spanish woman who has recently moved to Berlin finds her flirtation with a local guy turn potentially deadl